### Preparing Fragments 

In [1]:
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.Draw import IPythonConsole
import pandas as pd
from rdkit.Chem import rdMolDescriptors as rdescriptors
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sys    
import pubchempy as pcp
import warnings
warnings.filterwarnings('ignore')
from rdkit import RDLogger 
RDLogger.DisableLog('rdApp.*')
from sklearn.utils import shuffle
import sys
sys.path.insert(0,"./../../fip3")
from fip.profiles import CooccurrenceProfile
import fip.chem
from fip import profiles
from itertools import chain
from ast import literal_eval
import re

In [2]:
#Loading database from csv 
suppl_coco = pd.read_csv('../../data/CP1/COCONUT/COCOrandom.csv', delimiter = ' ')

In [3]:
suppl_coco

,Coconut_id,Smiles,MW,logP,Inchi,Inchi_s,Name,ECFP,Bitset
0,CNP0239046,CCCCCC1C(=O)CCC1CC(=O)OC,226.156895,2.72510,KVWWIYGFBYDJQC-UHFFFAOYSA-N,KVWWIYGFBYDJQC,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[80, 142, 195, 226, 294, 314, 334, 339, 370, 5..."
1,CNP0205086,Cc1c2c(cc3c1oc(=O)c1ccccc13)CN(c1cccc(Cl)c1)CO2,377.081871,5.26452,DWNKYEMDZRSBHK-UHFFFAOYSA-N,DWNKYEMDZRSBHK,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[133, 137, 165, 216, 252, 314, 350, 382, 398, ..."
2,CNP0361840,COc1cc(C2CC(=O)Oc3ccc4c(=O)c(-c5ccccc5)coc4c32...,458.136553,4.92680,ZRAWZNDFONIPGO-UHFFFAOYSA-N,ZRAWZNDFONIPGO,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[45, 60, 133, 145, 222, 227, 231, 249, 282, 28..."
3,CNP0264800,C=C1C(=O)OC2C=C(CO)CCC(OCC)C3=CC(OC3=O)C12,320.125988,1.05360,PTJPAEBQCZMNAV-UHFFFAOYSA-N,PTJPAEBQCZMNAV,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[35, 80, 176, 222, 294, 314, 408, 514, 650, 65..."
4,CNP0396333,CC(=O)C(CC=C(C)C)C(C)=CCOc1ccc(CCNC(=O)c2ccccc...,419.246044,5.54570,FGXBFPFGWRVMGG-UHFFFAOYSA-N,FGXBFPFGWRVMGG,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[1, 8, 80, 102, 197, 265, 310, 322, 367, 389, ..."
...,...,...,...,...,...,...,...,...,...
386292,CNP0315857,COC(=O)C1=COC(OC2OC(COC(=O)c3ccc(O)c(OC)c3)C(O...,554.163556,-0.63720,LIVSNGMPJUCRLO-UHFFFAOYSA-N,LIVSNGMPJUCRLO,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[58, 80, 145, 147, 167, 231, 233, 249, 314, 42..."
386293,CNP0032553,Cc1c(CCC(=O)NCC(=O)NCC(=O)[O-])c(=O)oc2cc(O)c(...,395.065152,-0.62468,SSZVGMXOKPZNCV-UHFFFAOYSA-M,SSZVGMXOKPZNCV,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[41, 80, 197, 252, 254, 305, 314, 366, 406, 42..."
386294,CNP0223192,CC(=O)Oc1c(C)cc2c(c1O)C(=O)C(C1=CC(=O)c3cc(C)c...,490.089997,2.87624,JODPOMYHTFZNTK-UHFFFAOYSA-N,JODPOMYHTFZNTK,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[16, 39, 84, 202, 224, 314, 333, 352, 516, 641..."
386295,CNP0356727,C[N+]([O-])=NCOC1OC(CO)C(O)C(OC2OC(COC3OC(CO)C...,576.201398,-6.99890,STHHNNUYQQUJGY-UHFFFAOYSA-N,STHHNNUYQQUJGY,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[77, 80, 170, 222, 349, 443, 489, 588, 610, 65..."


In [4]:
%%time
instance = []
for x in suppl_coco.Smiles:
    m = Chem.MolFromSmiles(x)
    z =fip.chem.rdmol2morgan_feature_smiles(m, radius=2, min_radius=2, all_bonds_explicit=False, canonical_smiles=True, isomeric_smiles=False, all_H_explicit=True)
    z = str(z)
    z = z.replace("{","")
    z = z.replace("}","")
    instance.append(z)

CPU times: user 1h 24min 48s, sys: 4.6 s, total: 1h 24min 53s
Wall time: 1h 24min 55s


In [5]:
suppl_coco['Fragments'] = instance

In [6]:
suppl_coco.to_csv('../../data/CPF/COCONUT/COCOrandomF.csv', sep=' ', index = False)

In [8]:
suppl_coco = pd.read_csv('../../data/CPF/COCONUT/COCOrandomF.csv', delimiter = ' ')

In [28]:
suppl_coco['Fragments'][1]

"'[c][c]([CH2])[cH][c]([c])[c]', '[cH][cH][cH][cH][c]', '[c][c]([O])[c]([CH3])[c]([c])[o]', '[c][cH][c]([Cl])[cH][cH]', '[cH][cH][cH][c]([cH])[Cl]', '[cH][cH][c]([c]([cH])[c])[c]([c])[cH]', '[c][o][c]([c]([c])[CH3])[c]([cH])[c]', '[cH][c]([N])[cH][c]([cH])[Cl]', '[c][cH][cH][cH][c]', '[c][cH][c]([c]([c])[cH])[c]([c])[o]', '[cH][cH][c]([c](=[O])[o])[c]([c])[cH]', '[c][o][c](=[O])[c]([cH])[c]', '[c][c](=[O])[o][c]([c])[c]', '[c][N]([CH2])[CH2][c]([c])[cH]', '[cH][cH][c]([cH][c])[N]([CH2])[CH2]', '[c][O][CH2][N]([c])[CH2]', '[c][cH][c]([CH2][N])[c]([c])[O]', '[c][c]([c])[O][CH2][N]', '[c][CH2][N]([CH2][O])[c]([cH])[cH]', '[cH][cH][cH][c]([cH])[N]', '[c][cH][cH][cH][cH]', '[cH][c]([CH2])[c]([O][CH2])[c]([c])[CH3]', '[cH][cH][cH][c]([c])[c]'"

In [27]:
instance[1]

"'[c][c]([CH2])[cH][c]([c])[c]', '[cH][cH][cH][cH][c]', '[c][c]([O])[c]([CH3])[c]([c])[o]', '[c][cH][c]([Cl])[cH][cH]', '[cH][cH][cH][c]([cH])[Cl]', '[cH][cH][c]([c]([cH])[c])[c]([c])[cH]', '[c][o][c]([c]([c])[CH3])[c]([cH])[c]', '[cH][c]([N])[cH][c]([cH])[Cl]', '[c][cH][cH][cH][c]', '[c][cH][c]([c]([c])[cH])[c]([c])[o]', '[cH][cH][c]([c](=[O])[o])[c]([c])[cH]', '[c][o][c](=[O])[c]([cH])[c]', '[c][c](=[O])[o][c]([c])[c]', '[c][N]([CH2])[CH2][c]([c])[cH]', '[cH][cH][c]([cH][c])[N]([CH2])[CH2]', '[c][O][CH2][N]([c])[CH2]', '[c][cH][c]([CH2][N])[c]([c])[O]', '[c][c]([c])[O][CH2][N]', '[c][CH2][N]([CH2][O])[c]([cH])[cH]', '[cH][cH][cH][c]([cH])[N]', '[c][cH][cH][cH][cH]', '[cH][c]([CH2])[c]([O][CH2])[c]([c])[CH3]', '[cH][cH][cH][c]([c])[c]'"